In [ ]:
import csv
import io
import os
import re

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "INSERT-YOUR-CREDENTIALS-HERE"

# Imports the Google Cloud client library
from google.cloud import vision

# Instantiates a client
client = vision.ImageAnnotatorClient()

# The path of the directory containing the images
images_path = os.path.abspath('../imgs')

# Read keywords from a list of dictionaries and get dictionary names
sub_directory = 'sub'
dict_filenames = [os.path.join(sub_directory, file) for file in os.listdir(sub_directory) if file.endswith(".txt")]
dict_names = [os.path.splitext(os.path.basename(filename))[0] for filename in dict_filenames]

# Load the list of processed images from a file
try:
    with open('imgProcessed.txt', 'r', encoding="utf-8-sig") as f:
        processed_images = [line.strip() for line in f.readlines()]
except FileNotFoundError:
    processed_images = []

# Open a CSV file to write the results
with open('imgLabels.csv', 'a', newline='', encoding="utf-8-sig") as labels_file:
    labels_writer = csv.writer(labels_file)

    # Write the header row for the labels file if the file is empty
    if os.stat('imgLabels.csv').st_size == 0:
        labels_writer.writerow(["Image", "Label", "Score"])

    # Loop through all the images in the directory
    for filename in os.listdir(images_path):
        # Skip any non-image files
        if not filename.endswith(".jpg") and not filename.endswith(".jpeg") and not filename.endswith(".png"):
            continue
        # Skip any images that have already been processed
        if filename in processed_images:
            continue
        processed_images.append(filename)
        
        # Load the image into memory
        file_path = os.path.join(images_path, filename)
        with io.open(file_path, 'rb') as image_file:
            content = image_file.read()
        image = vision.Image(content=content)
        
        # Perform label detection on the image
        response = client.label_detection(image=image)
        labels = set([label.description.lower() for label in response.label_annotations])

        # Write the results for each label to the labels file
        for label in labels:
            score = None
            for annotation in response.label_annotations:
                if label == annotation.description.lower():
                    score = annotation.score
                    break
            labels_writer.writerow([filename, label, score])

        # Save the updated list of processed images to the file
        with open('imgProcessed.txt', 'w', encoding="utf-8-sig") as f:
            for img in processed_images:
                f.write(f"{img}\n")
